In [21]:
# Importing all necessary libraries

import pandas as pd
import numpy as np
import cv2
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import progressbar
from alive_progress import alive_bar
import time
import os
import collections
import random


from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,InputLayer, Flatten
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Sequential, Model


from sklearn.metrics import multilabel_confusion_matrix



## count images in the folder, find duplicates if any 

In [2]:
folder=r'D:\Football_image_classification\IAUFD_dataset\IAUSD_all'
a=[]
for filename in os.listdir(folder):
    f=int(filename.split('.')[0])
    a.append(f)
a.sort()

print([item for item, count in collections.Counter(a).items() if count>1])   # check for repeated images
print(len(a))                                                                # check for 100k images

[]
100000


## reduce 10 label to 5 labels 

In [3]:
# read labels
labels=pd.read_excel(r"D:\Football_image_classification\IAUFD_dataset\IAUSD_Label.xlsx")
# rearrange column names and index 
labels.columns = labels.iloc[0]
labels.drop(index=labels.index[0], axis=0, inplace=True)
labels.reset_index()
labels=labels.filter(regex='(?i)^(?!NaN).+', axis=1)
# drop labels not needed for the project
labels.drop(columns=['referee Image','Start/Restar the game','Stadium View','Penalty Image', 'Free Kick'],inplace=True)
# convert object datatype to int
for k in labels.columns:
    labels[k]=labels[k].astype('str').astype('int')
    print(labels[k].value_counts())
    print("--"*7)


0     75991
1     24007
11        2
Name: Gate , dtype: int64
--------------
0    98625
1     1375
Name: The joy of the players, dtype: int64
--------------
0    99409
1      591
Name: yellow card, dtype: int64
--------------
0    99929
1       71
Name: Red Card, dtype: int64
--------------
0     52311
1     47687
11        2
Name: Ball, dtype: int64
--------------


In [4]:
labels[labels['Gate ']==11]

,Gate,The joy of the players,yellow card,Red Card,Ball
47590,11,0,0,0,1
64991,11,0,0,0,0


In [5]:
labels[labels['Ball']==11]

,Gate,The joy of the players,yellow card,Red Card,Ball
70222,0,0,0,0,11
86553,1,0,0,0,11


In [6]:
labels.loc[47590,'Gate ']=1   # change label 11 to 1
labels.loc[64991,'Gate ']=1
labels.loc[70222,'Ball']=1
labels.loc[86553,'Ball']=1
                              # remove images with no labels
b=labels.sum(axis=1)
b=b[b==0].index.tolist()
a1=list(set(a)-set(b))
a1.sort()
lbl=labels.loc[a1]
y1=lbl.iloc[10]               # multi-label image classification
list(y1),len(a1)

([1, 0, 0, 0, 1], 59276)

In [7]:
labels.tail(10)

,Gate,The joy of the players,yellow card,Red Card,Ball
99991,0,0,0,0,1
99992,1,0,0,0,1
99993,0,0,0,0,1
99994,0,0,0,0,1
99995,0,0,0,0,1
99996,0,0,0,0,1
99997,0,0,0,0,0
99998,0,0,0,0,0
99999,0,0,0,0,0
100000,0,0,0,0,0


In [8]:
lbl.loc[99992]

0
Gate                      1
The joy of the players    0
yellow card               0
Red Card                  0
Ball                      1
Name: 99992, dtype: int32

##  Resize and flatten the images  to 1 D, 2D, 3D

In [9]:
im_width=256//4
im_height=144//4
image=np.empty((1,im_width*im_height))
multi_lbl=np.empty((1,5))
 
with alive_bar(len(a1), force_tty=True, title='Processing..') as bar:
    for filename in os.listdir(folder):
        f=int(filename.split('.')[0])
        if f in a1:
            y1=list(lbl.loc[f])
            img=cv2.imread(folder+'\\'+filename, 0)  # reading image in grey scale(Folder path and image name )
            im_resize = cv2.resize(img, (im_width, im_height))
            im_flatten=im_resize.flatten()
            image=np.vstack((image,im_flatten))
            multi_lbl=np.vstack((multi_lbl,y1))
            bar()
        
        
print(image.shape,multi_lbl.shape)
np.save("ImageData1D", image)
np.save("LabelD",multi_lbl)

Processing.. |████████████████████████████████████████⚠︎ (!) 59276/59277 [100%] in 3:23:25.7 (4.86/s)                    
(59277, 2304) (59277, 5)


In [13]:
im_width=256//4
im_height=144//4
imageGrey=np.zeros((len(a1),im_height,im_width))
multi_lblgrey=np.zeros((len(a1),5))
count=0
bar=progressbar.ProgressBar(max_value=len(a1)).start()
for filename in os.listdir(folder):
    f=int(filename.split('.')[0])
    if f in a1:
        y1=list(lbl.loc[f])
        img=cv2.imread(folder+'\\'+filename,0)            # reading image in grey scale(Folder path and image name )
        im_resize = cv2.resize(img, (im_width,im_height))
        im_reshape=im_resize.reshape(-1,im_height,im_width)
        imageGrey[count,:,:]=im_reshape
        multi_lblgrey[count,:]=y1
        count+=1
        bar.update(count)
       
       
print(image.shape,multi_lbl.shape)
print(imageGrey.shape,multi_lblgrey.shape)
np.save("ImageData2D", imageGrey)
np.save("Labe2D",multi_lblgrey)

 99% (59272 of 59276) |################# | Elapsed Time: 0:02:33 ETA:   0:00:00

(59277, 2304) (59277, 5)
(59276, 36, 64) (59276, 5)


In [14]:
im_width=256//4
im_height=144//4
imageColor=np.zeros((len(a1),im_height,im_width,3))
multi_lblcolor=np.zeros((len(a1),5))
count=0
bar=progressbar.ProgressBar(max_value=len(a1)).start()
for filename in os.listdir(folder):
    f=int(filename.split('.')[0])
    if f in a1:
        y1=list(lbl.loc[f])
        img=cv2.imread(folder+'\\'+filename, 1)            # reading image (Folder path and image name )
        im_resize = cv2.resize(img, (im_width,im_height))
        im_reshape=im_resize.reshape(-1,im_height,im_width,3)
        imageColor[count,:,:,:]=im_reshape
        multi_lblcolor[count,:]=y1
        count+=1
        bar.update(count)
np.save("ImageData3D", imageColor)
np.save("Labe3D",multi_lblcolor)       

 99% (59273 of 59276) |################# | Elapsed Time: 0:07:09 ETA:   0:00:00